# KNN with Parameter Tuning

In this project, I implement from scratch the k-NN algorithm on a dataset containing breast cancer data. 
I also implement my own cross-validation algorithm in order to tune the model.

In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math 
import statistics

In [43]:
breast_cancer_train = pd.read_csv("breast_cancer_train.csv")
breast_cancer_test = pd.read_csv("breast_cancer_test.csv")

breast_cancer_train['target'].value_counts(normalize=1)

x_train = breast_cancer_train.drop('target', axis=1)
x_test = breast_cancer_test.drop('target', axis=1)

y_train = breast_cancer_train[['target']]
y_test = breast_cancer_test[['target']]

x_train_normalized = ((x_train - x_train.min()) / (x_train.max() - x_train.min()) )
x_test_normalized = ((x_test - x_test.min()) / (x_test.max() - x_test.min()) )

In [44]:
def euclidean_distance(p, q):
    sum_cathetus = 0
    for i in range(0, len(p)):
        cathetus = (p[i] - q[i])**2
        sum_cathetus += cathetus
    
    return math.sqrt(sum_cathetus)

    
def knn(x_train_normalized, y_train, query_point, k, distance_func):
    distances = []
    for index in range(0, len(x_train_normalized)):
        d = distance_func(x_train_normalized.iloc[index], query_point)
        distances.append((d,index))
    sorted(distances)

    k_nearest_neighbors = distances[0:k]

    labels = []
    for neighbor in k_nearest_neighbors:
        index = neighbor[1]
        label = y_train['target'][index]
        labels.append(label)
    prediction = statistics.mode(labels)

    return prediction

### Evaluating k = 31 with Euclidean Distance

In [47]:
predictions = []

for i in range (0, len(x_test)):
    prediction = knn(x_train_normalized, y_train, x_test.iloc[i], 31, euclidean_distance)
    predictions.append(prediction)

y_test['prediction'] = predictions

C:\Users\rodri\AppData\Local\Temp\ipykernel_21112\1831007801.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_test['prediction'] = predictions
